In [2]:
import findspark
findspark.init()

In [3]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.ml.feature import PCA, VectorAssembler
from pyspark.ml.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.util import MLUtils
import numpy as np
from pyspark.ml.feature import StandardScaler
import pyspark.sql.functions as f
import pyspark.sql.types
import pandas as pd
from pyspark.sql import Row

In [4]:
spark = SparkSession.builder.getOrCreate()
sc = SparkContext.getOrCreate()

In [5]:
trans_data = spark.read.csv("New_Aggregated_data_final.csv", inferSchema=True, header=True)

In [6]:
customer_data = spark.read.csv("Customer_data1.csv", inferSchema=True, header=True)

In [7]:
trans_data = trans_data.withColumn("sum_prev_day_onl", trans_data["sum_prev_day_onl"].cast("integer"))
trans_data = trans_data.withColumn("sum_prev_day_mon_onl", trans_data["sum_prev_day_mon_onl"].cast("integer"))

In [8]:
trans_data1 = trans_data.select('amt', 'Balance',
      'sum_prev_day', 'cnt_prev_day_onl', 'sum_prev_day_onl',
       '24hrsAvg','qtrAvg','wkAvg', 'monAvg','yrAvg').fillna(0)

In [9]:
# merging the data together by their unique "id"
train = trans_data.join(customer_data,how='left',on='cc_num')
# all_data.show()

In [10]:
# from pyspark.sql.functions import *
# train.where(col('cc_num').isNull()).count()
# df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()
# customer_data.select([count(when(col(c).isNull(),c)).alias(c) for c in customer_data.columns]).show()

### PCA

In [11]:
def vectorizedf(inputdf,vars='all'):
        """Returns the input spark dataframe with an additional column of dense vector features"""
        if vars=='all':
            myfeatures = inputdf.columns
        else:
            myfeatures=vars
        
        # assemble features
        assembler = VectorAssembler(inputCols = myfeatures,outputCol="features")
        assembled = assembler.transform(inputdf).select('features')
        
        # scale and standardize data
        scaler = StandardScaler(inputCol='features', outputCol='scaledFeatures', withStd=True, withMean=True)
        scaled = scaler.fit(assembled).transform(assembled)

        return myfeatures, scaled

def estimateCovariance(df):
    """Compute the covariance matrix for a given dataframe.

    Note:
        The multi-dimensional covariance array should be calculated using outer products.  Don't
        forget to normalize the data by first subtracting the mean.

    Args:
        df:  A Spark dataframe with a column named 'scaledFeatures', which (column) consists of DenseVectors.

    Returns:
        np.ndarray: A multi-dimensional array where the number of rows and columns both equal the
            length of the arrays in the input dataframe.
    """
    
    dfZeroMean = df.select('scaledFeatures')

    return dfZeroMean.rdd.map(lambda x: np.outer(x,x)).sum()/df.count()

In [12]:
from numpy.linalg import eigh

def computeEigVals(df):
    """Computes all eigenvalues.

    Note:
        All eigenvalues should be returned in sorted order (largest to smallest). `eigh` returns
        each eigenvectors as a column. 

    Args:
        df: A Spark dataframe with a 'features' column, which (column) consists of DenseVectors.

    Returns:
        Eigenvalues is an array of length d (the number of features).
     """
    dfT = vectorizedf(df)[1]
    cov = estimateCovariance(dfT)
    col = cov.shape[1]
    eigVals, eigVecs = eigh(cov)
    inds = np.argsort(eigVals)
    eigVecs = eigVecs.T[inds[-1:-(col+1):-1]]  
    eigVals = eigVals[inds[-1:-(col+1):-1]]  # sort eigenvals

    return eigVals

In [13]:
def computePCA(df):
    """
    Compute Principal components using pyspark.ml inbuilt function
    
    Args: 
        df: A Spark dataframe
        k: The number of principal components to be computed which must be less than or equal
            to the number of variables in the dataframe
            
    Returns:
        Spark dataframe with contributions of each variable to each principal component. Columns
        are principal components, rows are variable names
    
    """
    # compute PCA
    pca = PCA(k=10, inputCol='scaledFeatures', outputCol='pcaFeatures')
    cols, dfT = vectorizedf(df)
    model = pca.fit(dfT)
    transformed_feature = model.transform(dfT)
    
    # compute number of significant principal components
    totalpercent = np.cumsum(model.explainedVariance.toArray())
    numsigpcs = np.where(totalpercent > 0.85)[0][0]
    
    # create dataframe of mapping
    pcs = np.round(model.pc.toArray(),4)[:,:numsigpcs]
#     df_pc = pd.DataFrame(pcs, columns = ['PC'+str(i) for i in range(1, len(cols)+1)], index = cols)
    
    return cols, numsigpcs, pcs

In [14]:
def extractImpVars(n, pcs, eigVals, cols):
    """
    Order the results from the PCA computation to find the variables with the greatest contributions
    
    Args:
        n: number of significant principal components from pca computation (85%)
        pcs: computed principal components
        eigVals: computed eigenvalues of the principal components
        cols: list of columns used to compute  principal components
        
    Returns:
        SigVars: a list of the most significant variables
    """
    contrib = (pcs[:,:n]*eigVals[:n]).sum(axis=1)
    
    contribdf = pd.DataFrame(contrib, columns = ['PC'], index = cols)
    
    impVars = contribdf.sort_values(by=['PC'], ascending = False).index[:4].tolist()
    
    return impVars    
    

In [15]:
def pcaColumnSelector(df):
    """
    Returns a dataframe with top columns selected by performing PCA
    """
    eigVals = computeEigVals(trans_data1)
    
    cols, n, pcs = computePCA(trans_data1)
    
    impVars = extractImpVars(n, pcs, eigVals, cols)
    
    return df.select(impVars)

In [16]:
pcData = pcaColumnSelector(trans_data1)
pcData.columns

['Balance', 'amt', 'cnt_prev_day_onl', 'sum_prev_day']

### Feature Engineering

In [17]:
from pyspark.sql.functions import *
from pyspark.sql.functions import unix_timestamp

In [18]:
train = train.withColumn("Birthdate",from_unixtime(unix_timestamp(train['dob'], 'MM/dd/yyyy')))

In [19]:
train=(train.withColumn('Yearofbirth',year(train['Birthdate'])))

In [20]:
train = train.withColumn("trans_date",from_unixtime(unix_timestamp(train['trans_date'], 'MM/dd/yyyy')))

In [21]:
train=(train.withColumn('Month',month(train['fulltime'])))
train = train.withColumn("Time",hour(train["fulltime"]))

In [22]:
train = train.withColumn('today_date',lit(2019))
train = train.withColumn('Age',train['today_date']-train['Yearofbirth'])
train.select('today_date','Age','dob').show(5)

+----------+---+---------+
|today_date|Age|      dob|
+----------+---+---------+
|      2019| 45|9/23/1974|
|      2019| 45|9/23/1974|
|      2019| 45|9/23/1974|
|      2019| 45|9/23/1974|
|      2019| 45|9/23/1974|
+----------+---+---------+
only showing top 5 rows



In [23]:
#spark.conf.set("spark.sql.execution.arrow.fallback.enabled", "false")
a=train.drop('first','last')
train2 = a.toPandas()

In [24]:
train2["fulltimepd"] =  pd.to_datetime(train2['unix_time'],unit='s')

train2["Weekday"] = train2["fulltimepd"].dt.strftime("%A")

In [25]:
#train["Time_short_for_grouping"] = train["Time_short_for_grouping"].astype(int)
bins = [2,6,11,18,22]
labels = ["Early Morning","Morning","Afternoon","Evening"]
train2["Time of day"] = pd.cut(train2.Time,bins=bins,labels=labels)
train2["Time of day"]=train2["Time of day"].cat.add_categories('Midnight') 
train2["Time of day"] = train2["Time of day"].fillna('Midnight')

In [26]:
train2["Month"] = train2["fulltimepd"].dt.strftime("%B")

In [27]:
def haversine_(lat1, lng1, lat2, lng2):
    """function to calculate haversine distance between two co-ordinates"""
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    AVG_EARTH_RADIUS = 6371  # in km
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
    h = 2 * AVG_EARTH_RADIUS * np.arcsin(np.sqrt(d))
    return(h)

In [28]:
def manhattan_distance_pd(lat1, lng1, lat2, lng2):
    """function to calculate manhatten distance between pick_drop"""
    a = haversine_(lat1, lng1, lat1, lng2)
    b = haversine_(lat1, lng1, lat2, lng1)
    return a + b

In [29]:
train2['hvsine']= haversine_(train2['lat'].values,
                                 train2['long'].values, train2['merch_lat'].values,
                                             train2['merch_long'].values)

In [30]:
train2['mnhtn']= manhattan_distance_pd(train2['lat'].values,
                                 train2['long'].values, train2['merch_lat'].values,
                                             train2['merch_long'].values)

In [31]:
train2['hvsine2']= haversine_(train2['prev_lat'].values,
                                 train2['prev_long'].values, train2['merch_lat'].values,
                                             train2['merch_long'].values)

C:\Users\aoladipo001\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel\__main__.py:8: RuntimeWarning: invalid value encountered in arcsin


In [32]:
train2['mnhtn2']= manhattan_distance_pd(train2['prev_lat'].values,
                                 train2['prev_long'].values, train2['merch_lat'].values,
                                             train2['merch_long'].values)

C:\Users\aoladipo001\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel\__main__.py:8: RuntimeWarning: invalid value encountered in arcsin


In [33]:
#Distance and time dfference
train2['distandtime1'] = train2['mnhtn2']/train2['time_diff_min']
# train2['distandtime2'] = train2['hvsine2']/train2['time_diff_min']
# train['distandtime3'] = train['bearing2']/train['time_diff_min']

#train2['distandtime'] =train2['manhtn']/train2['time_diff_min']

In [34]:
# Amount versus year average
train2['amt_yrAvg'] = train2['amt']/train2['yrAvg']

In [35]:
#
train3=train2.copy(deep=True)

In [36]:
trainimp_f = train3[['Channel', 'Transaction Type', 'gender', 'amt', 'Balance',
       'Month', 'Weekday','Time of day','Age','mnhtn','sum_prev_day', 'cnt_prev_day_onl', 'sum_prev_day_onl',
       '24hrsAvg','qtrAvg','wkAvg', 'monAvg','yrAvg','mnhtn2','distandtime1','amt_yrAvg','is_fraud']]

In [37]:
trainimp_f.isnull().sum()

Channel                 0
Transaction Type        0
gender                  0
amt                     0
Balance                 0
Month                   0
Weekday                 0
Time of day             0
Age                     0
mnhtn                   0
sum_prev_day         1496
cnt_prev_day_onl    15761
sum_prev_day_onl    17759
24hrsAvg             1496
qtrAvg                332
wkAvg                 332
monAvg                332
yrAvg                 332
mnhtn2                 98
distandtime1           98
amt_yrAvg             332
is_fraud                0
dtype: int64

In [38]:
colsna= ['sum_prev_day','cnt_prev_day_onl','sum_prev_day_onl',
         '24hrsAvg','qtrAvg','wkAvg','monAvg','yrAvg','mnhtn2','distandtime1','amt_yrAvg']

In [39]:
for col in colsna:
    trainimp_f[col] = trainimp_f[col].fillna(0)

C:\Users\aoladipo001\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [40]:
trainimp_f.head()

,Channel,Transaction Type,gender,amt,Balance,Month,Weekday,Time of day,Age,mnhtn,...,sum_prev_day_onl,24hrsAvg,qtrAvg,wkAvg,monAvg,yrAvg,mnhtn2,distandtime1,amt_yrAvg,is_fraud
0,POS,POS,F,95,942,January,Friday,Midnight,45,1.688628,...,0.0,0.00,0.00,0.00,0.00,0.0,0.000000,0.000000,0.000000,0
1,POS,POS,F,90,0,January,Friday,Evening,45,2.709298,...,0.0,0.00,0.00,0.00,0.00,0.0,1.020535,0.001001,0.000000,0
2,Web,Online,F,188,6746,January,Friday,Evening,45,1.880668,...,0.0,0.00,0.00,0.00,0.00,0.0,2.175526,0.022514,0.000000,0
3,Mobile,USSD,F,100,133,January,Saturday,Afternoon,45,1.911466,...,0.0,124.33,124.33,124.33,124.33,124.0,3.791882,0.003087,0.806452,0
4,ATM,ATM,F,79,3115,January,Sunday,Morning,45,1.804176,...,0.0,100.00,118.25,118.25,118.25,118.0,1.529470,0.001447,0.669492,0


In [41]:
from pyspark.sql.types import (StructField, StringType,
                              IntegerType, StructType, DoubleType)

In [42]:
data_schema = StructType([StructField("Channel", StringType(), True),
               StructField("Transaction Type", StringType(), True),
               StructField("gender", StringType(), True),
               StructField("amt", IntegerType(), True),
               StructField("Balance", IntegerType(), True),
               StructField("Month", StringType(), True),
               StructField("Weekday", StringType(), True),
               StructField("Time of day", StringType(), True),
               StructField("Age", IntegerType(), True),
               StructField("mnhtn",  DoubleType(), True),
               StructField("sum_prev_day", DoubleType(), True),
               StructField("sum_prev_day_onl", DoubleType(), True),
               StructField("cnt_prev_day_onl", DoubleType(), True),
               StructField("24hrsAvg", DoubleType(), True),
               StructField("wkAvg", DoubleType(), True),
               StructField("monAvg", DoubleType(), True),
               StructField("qtrAvg", DoubleType(), True),
               StructField("yrAvg", DoubleType(), True),
               StructField("mnhtn2", DoubleType(), True),
               StructField("distandtime1", DoubleType(), True),
               StructField("amt_yrAvg", DoubleType(), True),
               StructField("is_fraud", IntegerType(), True)])

In [43]:
#Convert
#spark.conf.set("spark.sql.execution.arrow.fallback.enabled", "false")
#from pyspark.sql.types import *
#from pyspark.sql import SqlContext
trainimp_f = spark.createDataFrame(trainimp_f,schema=data_schema)

In [44]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline

### MLlib Transformer Pipeline

In [45]:
def getFeatures(numericCols, categoricalCols):
    """
    Get dummy variables and concatenate with continous variables
    
    Args:
        df: the dataframe
        numericCols: list of columns with numerical data
        categoricalCols: list of columns with categorical data
        labelCol: target variable column name
    
    Returns:
        featurePipeline: pipeline object to extract features
    
    """
    from pyspark.ml import Pipeline
    from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
    from pyspark.sql.functions import col
    
    indexers = [ StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c))
                    for c in categoricalCols ]
    
    encoders = [ OneHotEncoder(inputCol=indexer.getOutputCol(),
                              outputCol="{0}_encoded".format(indexer.getOutputCol()))
                    for indexer in indexers ]
    
    assembler_indexed = VectorAssembler(inputCols= [indexer.getOutputCol() for indexer in indexers]
                                + numericCols, 
                                outputCol='features_ind')

    assembler_encoded = VectorAssembler(inputCols= [encoder.getOutputCol() for encoder in encoders]
                                + numericCols, 
                                outputCol='features_enc')
    
    featurePipeline = Pipeline(stages=indexers+encoders+[assembler_indexed,assembler_encoded])
    
    return featurePipeline

In [46]:
def getModels(indexedFeatures, encodedFeatures, labelCol):
    """
    Get instances of models for the ensemble model
    
    Args:
        indexedFeatures: name of indexed features column
        encodedFeatures: name of encoded features column
        labelCol: target variable column name
    """
    
    from pyspark.ml import Pipeline
    from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, LogisticRegression
    
    rfc = RandomForestClassifier(labelCol=labelCol, predictionCol='rf_prediction', 
                                 probabilityCol='rf_probability', rawPredictionCol='rf_rawPrediction',
                                 featuresCol= indexedFeatures,
                                 maxDepth = 11,numTrees=40,seed=1)    

    gbt = GBTClassifier(labelCol=labelCol,  featuresCol= indexedFeatures,
                        predictionCol='gbt_prediction', maxIter=33, 
                        maxDepth = 3,stepSize=0.72,seed=1 )

    logr = LogisticRegression(featuresCol=encodedFeatures, labelCol=labelCol, predictionCol='logr_prediction', 
                                 probabilityCol='logr_probability', rawPredictionCol='logr_rawPrediction')

    modelPipeline = Pipeline(stages=[rfc,gbt,logr])
    
    return modelPipeline

In [47]:
numCols = ['Balance', 'amt', 'cnt_prev_day_onl', 'sum_prev_day']
catCols = []

# pipeline.fit(train_data).transform(train_data)

In [48]:
from pyspark.ml import Pipeline
finalPipeline = Pipeline(stages=[getFeatures(numericCols=numCols, categoricalCols=catCols),
                                getModels(indexedFeatures='features_ind', encodedFeatures='features_enc',
                                         labelCol='is_fraud')])

In [49]:
train_data,test_data = trainimp_f.randomSplit([0.7,0.3])

In [50]:
mod = finalPipeline.fit(train_data)

In [51]:
predall = mod.transform(test_data)

In [108]:
np.average([1,2,3])

2.0

In [137]:
from pyspark.sql.functions import explode, col, avg, mean
from pyspark.sql.types import FloatType, ArrayType
from pyspark.ml.linalg import VectorUDT

getallprobs = udf(lambda a,b,c:(float(a[1])+float(b[1])+float(c[1]))/3, FloatType())
# getallprobs = udf(lambda a:[float(i[1]) for i in a], ArrayType(FloatType()))


getprob = udf(lambda v:np.average(np.array(v)), FloatType())

# predall.select(getallprobs('probability','rf_probability','logr_probability').alias('prob')).show(truncate=False)
# predall.select(getprob('probability')).show(truncate=False)
a = predall.withColumn('prob', getallprobs('probability','rf_probability','logr_probability'))

In [138]:
a.select('prob','probability','rf_probability','logr_probability').show(truncate=False)

+------------+------------------------------------------+------------------------------------------+------------------------------------------+
|prob        |probability                               |rf_probability                            |logr_probability                          |
+------------+------------------------------------------+------------------------------------------+------------------------------------------+
|0.010777077 |[0.9922696197734491,0.007730380226550926] |[0.991882734040016,0.008117265959984084]  |[0.9835164154458468,0.01648358455415326]  |
|0.003923363 |[0.994106867722186,0.005893132277813962]  |[0.99873907229826,0.0012609277017399565]  |[0.9953839709344245,0.004616029065575404] |
|0.0072181574|[0.9936664742383315,0.006333525761668457] |[0.9992305808225312,7.69419177468709E-4]  |[0.9854484726077202,0.01455152739227993]  |
|0.007158044 |[0.9936664742383315,0.006333525761668457] |[0.998276650492053,0.0017233495079470667] |[0.986582743736797,0.013417256263203

In [124]:
getallprobs = udf(lambda a,b,c:[float(a[1]),float(b[1]),float(c[1])], ArrayType(FloatType()))

StructType(List(StructField(Channel,StringType,true),StructField(Transaction Type,StringType,true),StructField(gender,StringType,true),StructField(amt,IntegerType,true),StructField(Balance,IntegerType,true),StructField(Month,StringType,true),StructField(Weekday,StringType,true),StructField(Time of day,StringType,true),StructField(Age,IntegerType,true),StructField(mnhtn,DoubleType,true),StructField(sum_prev_day,DoubleType,true),StructField(sum_prev_day_onl,DoubleType,true),StructField(cnt_prev_day_onl,DoubleType,true),StructField(24hrsAvg,DoubleType,true),StructField(wkAvg,DoubleType,true),StructField(monAvg,DoubleType,true),StructField(qtrAvg,DoubleType,true),StructField(yrAvg,DoubleType,true),StructField(mnhtn2,DoubleType,true),StructField(distandtime1,DoubleType,true),StructField(amt_yrAvg,DoubleType,true),StructField(is_fraud,IntegerType,true),StructField(features_ind,VectorUDT,true),StructField(features_enc,VectorUDT,true),StructField(rf_rawPrediction,VectorUDT,true),StructField(rf

In [77]:
predall.schema

StructType(List(StructField(Channel,StringType,true),StructField(Transaction Type,StringType,true),StructField(gender,StringType,true),StructField(amt,IntegerType,true),StructField(Balance,IntegerType,true),StructField(Month,StringType,true),StructField(Weekday,StringType,true),StructField(Time of day,StringType,true),StructField(Age,IntegerType,true),StructField(mnhtn,DoubleType,true),StructField(sum_prev_day,DoubleType,true),StructField(sum_prev_day_onl,DoubleType,true),StructField(cnt_prev_day_onl,DoubleType,true),StructField(24hrsAvg,DoubleType,true),StructField(wkAvg,DoubleType,true),StructField(monAvg,DoubleType,true),StructField(qtrAvg,DoubleType,true),StructField(yrAvg,DoubleType,true),StructField(mnhtn2,DoubleType,true),StructField(distandtime1,DoubleType,true),StructField(amt_yrAvg,DoubleType,true),StructField(is_fraud,IntegerType,true),StructField(features_ind,VectorUDT,true),StructField(features_enc,VectorUDT,true),StructField(rf_rawPrediction,VectorUDT,true),StructField(rf

In [112]:
predall.select('rf_probability','probability', 'logr_probability','rf_prediction').show(5,truncate=False)

+-----------------------------------------+-----------------------------------------+------------------------------------------+-------------+
|rf_probability                           |probability                              |logr_probability                          |rf_prediction|
+-----------------------------------------+-----------------------------------------+------------------------------------------+-------------+
|[0.984370079582342,0.015629920417657994] |[0.9904998854522721,0.009500114547727923]|[0.9821764167999824,0.017823583200017666] |0.0          |
|[0.9978892399481978,0.002110760051802235]|[0.9902339846618946,0.009766015338105394]|[0.9976713570549718,0.002328642945028161] |0.0          |
|[0.9991794471118405,8.20552888159403E-4] |[0.9935417822374103,0.006458217762589702]|[0.9949527214629974,0.005047278537002549] |0.0          |
|[0.988884177108617,0.011115822891382937] |[0.9902339846618946,0.009766015338105394]|[0.9977282693504353,0.0022717306495645957]|0.0          |

In [62]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

my_binary_eval = BinaryClassificationEvaluator(labelCol='is_fraud',
                                               rawPredictionCol='rf_rawPrediction')
print(my_binary_eval.evaluate(predall))

0.9789487755604309


In [113]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

my_binary_eval = BinaryClassificationEvaluator(labelCol='is_fraud',
                                               rawPredictionCol='rf_rawPrediction')
print(my_binary_eval.evaluate(predall))

0.9328404058860955


In [56]:
rfc_preds3.where('probability[1] > 0.5').select('rawPrediction','probability','prediction').show(truncate=False)

AnalysisException: "Can't extract value from probability#1574: need struct type but got vector;"

In [72]:
my_binary_eval = BinaryClassificationEvaluator(labelCol='is_fraud')
print(my_binary_eval.evaluate(rfc_preds))

0.9995282126010989


### Model Evaluation

In [73]:
tp = rfc_preds[(rfc_preds.is_fraud == 1) & (rfc_preds.prediction == 1)].count()
tn = rfc_preds[(rfc_preds.is_fraud == 0) & (rfc_preds.prediction == 0)].count()
fp = rfc_preds[(rfc_preds.is_fraud == 0) & (rfc_preds.prediction == 1)].count()
fn = rfc_preds[(rfc_preds.is_fraud == 1) & (rfc_preds.prediction == 0)].count()
print ("True Positives:", tp)
print ("True Negatives:", tn)
print ("False Positives:", fp)
print ("False Negatives:", fn)
print ("Total", rfc_preds.count())

True Positives: 170
True Negatives: 6058
False Positives: 19
False Negatives: 3
Total 6250


In [54]:
tp = gbt_preds[(gbt_preds.is_fraud == 1) & (gbt_preds.prediction == 1)].count()
tn = gbt_preds[(gbt_preds.is_fraud == 0) & (gbt_preds.prediction == 0)].count()
fp = gbt_preds[(gbt_preds.is_fraud == 0) & (gbt_preds.prediction == 1)].count()
fn = gbt_preds[(gbt_preds.is_fraud == 1) & (gbt_preds.prediction == 0)].count()
print ("True Positives:", tp)
print ("True Negatives:", tn)
print ("False Positives:", fp)
print ("False Negatives:", fn)
print ("Total", gbt_preds.count())

True Positives: 156
True Negatives: 6065
False Positives: 5
False Negatives: 19
Total 6245


### Saving & Loading Models

In [55]:
rfc_model.write().overwrite().save("models/rfc_model")

In [56]:
type(rfc_model)

pyspark.ml.classification.RandomForestClassificationModel

In [70]:
from pyspark.ml.feature import IndexToString, StringIndexer

df = spark.createDataFrame(
    [(0, "a"), (1, "b"), (2, "c"), (3, "a"), (4, "a"), (5, "c")],
    ["id", "category"])

indexer = StringIndexer(inputCol="category", outputCol="categoryIndex")
model = indexer.fit(df)
indexed = model.transform(df)

print("Transformed string column '%s' to indexed column '%s'"
      % (indexer.getInputCol(), indexer.getOutputCol()))
indexed.show()

print("StringIndexer will store labels in output column metadata\n")

converter = IndexToString(inputCol="categoryIndex", outputCol="originalCategory")
converted = converter.transform(indexed)

print("Transformed indexed column '%s' back to original string column '%s' using "
      "labels in metadata" % (converter.getInputCol(), converter.getOutputCol()))
converted.select("id", "categoryIndex", "originalCategory").show()

Transformed string column 'category' to indexed column 'categoryIndex'
+---+--------+-------------+
| id|category|categoryIndex|
+---+--------+-------------+
|  0|       a|          0.0|
|  1|       b|          2.0|
|  2|       c|          1.0|
|  3|       a|          0.0|
|  4|       a|          0.0|
|  5|       c|          1.0|
+---+--------+-------------+

StringIndexer will store labels in output column metadata

Transformed indexed column 'categoryIndex' back to original string column 'originalCategory' using labels in metadata
+---+-------------+----------------+
| id|categoryIndex|originalCategory|
+---+-------------+----------------+
|  0|          0.0|               a|
|  1|          2.0|               b|
|  2|          1.0|               c|
|  3|          0.0|               a|
|  4|          0.0|               a|
|  5|          1.0|               c|
+---+-------------+----------------+

